##  Perception for Binary classification

#### Loading the required libraries

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

import matplotlib.pyplot as plt

In [1]:
import keras

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Loading the data

In [3]:
data = pd.read_csv("BackOrders.csv",header=0)

#### Understand the Data

In [4]:
#Check number of rows and columns
data.shape

(10000, 23)

In [5]:
#Display columns
data.columns

Index(['sku', 'national_inv', 'lead_time', 'in_transit_qty',
       'forecast_3_month', 'forecast_6_month', 'forecast_9_month',
       'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month',
       'min_bank', 'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')

In [6]:
#Display index
data.index

RangeIndex(start=0, stop=10000, step=1)

See the top rows of the data

In [7]:
# Check top 'n' rows
data[:3]

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,1888279,117,NaN,0,0,0,0,0,0,15,...,0,-99.00,-99.00,0,No,No,Yes,Yes,No,No
1,1870557,7,2.0,0,0,0,0,0,0,0,...,0,0.50,0.28,0,Yes,No,No,Yes,No,No
2,1475481,258,15.0,10,10,77,184,46,132,256,...,0,0.54,0.70,0,No,No,No,Yes,No,No


In [8]:
#Summary statistics of each column
data.describe()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty
count,1.000000e+04,10000.000000,9433.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,2.046242e+06,267.388700,7.561645,26.115800,137.912800,248.780900,355.819500,36.492100,108.298300,209.889800,314.248600,36.063200,1.567400,-6.284197,-5.869804,1.716900
std,6.590184e+05,3937.266534,6.470560,340.288412,1458.350464,2511.049468,3425.324204,288.688666,801.982281,1515.773389,2200.196573,304.328827,27.482543,25.575629,24.859075,44.798882
min,1.111652e+06,-2999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-99.000000,-99.000000,0.000000
25%,1.510146e+06,2.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.630000,0.640000,0.000000
50%,1.898042e+06,10.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,4.000000,6.000000,0.000000,0.000000,0.820000,0.800000,0.000000
75%,2.811447e+06,55.000000,8.000000,0.000000,12.000000,25.000000,37.000000,5.000000,17.000000,34.000000,49.000000,3.000000,0.000000,0.960000,0.950000,0.000000
max,3.284750e+06,364065.000000,52.000000,20047.000000,98548.000000,169720.000000,214848.000000,10698.000000,35076.000000,67129.000000,89727.000000,16642.000000,1488.000000,1.000000,1.000000,2999.000000


Display data type of each variable

In [9]:
# Check data type of each attribute
data.dtypes

sku                    int64
national_inv           int64
lead_time            float64
in_transit_qty         int64
forecast_3_month       int64
forecast_6_month       int64
forecast_9_month       int64
sales_1_month          int64
sales_3_month          int64
sales_6_month          int64
sales_9_month          int64
min_bank               int64
potential_issue       object
pieces_past_due        int64
perf_6_month_avg     float64
perf_12_month_avg    float64
local_bo_qty           int64
deck_risk             object
oe_constraint         object
ppap_risk             object
stop_auto_buy         object
rev_stop              object
went_on_backorder     object
dtype: object

#### Observations

    sku is Categorical but is interpreted as int64 
    potential_issue, deck_risk, oe_constraint, ppap_risk, stop_auto_buy, rev_stop, and went_on_backorder are also categorical but is interpreted as object. 

#### Convert all the attributes to appropriate type

Data type conversion

    Using astype('category') to convert potential_issue, deck_risk, oe_constraint, ppap_risk, stop_auto_buy, rev_stop, and went_on_backorder attributes to categorical attributes.


In [10]:
for col in ['sku', 'potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']:
    data[col] = data[col].astype('category')

Display data type of each variable

In [11]:
#Display data type of each variable
data.dtypes

sku                  category
national_inv            int64
lead_time             float64
in_transit_qty          int64
forecast_3_month        int64
forecast_6_month        int64
forecast_9_month        int64
sales_1_month           int64
sales_3_month           int64
sales_6_month           int64
sales_9_month           int64
min_bank                int64
potential_issue      category
pieces_past_due         int64
perf_6_month_avg      float64
perf_12_month_avg     float64
local_bo_qty            int64
deck_risk            category
oe_constraint        category
ppap_risk            category
stop_auto_buy        category
rev_stop             category
went_on_backorder    category
dtype: object

#### Delete sku attribute

In [12]:
np.size(np.unique(data.sku, return_counts=True)[0])

10000

In [13]:
data.drop('sku', axis=1, inplace=True)

#### Missing Data

    Missing value analysis and dropping the records with missing values

In [14]:
data.isnull().sum()

national_inv           0
lead_time            567
in_transit_qty         0
forecast_3_month       0
forecast_6_month       0
forecast_9_month       0
sales_1_month          0
sales_3_month          0
sales_6_month          0
sales_9_month          0
min_bank               0
potential_issue        0
pieces_past_due        0
perf_6_month_avg       0
perf_12_month_avg      0
local_bo_qty           0
deck_risk              0
oe_constraint          0
ppap_risk              0
stop_auto_buy          0
rev_stop               0
went_on_backorder      0
dtype: int64

Observing the number of records before and after missing value records removal

In [15]:
print (data.shape)

(10000, 22)


In [16]:
#Since the number of missing values is about 5%. For initial analysis we ignore all these records
data = data.dropna(axis=0)

In [17]:
data.isnull().sum()
print(data.shape)

(9433, 22)


#### Converting Categorical to Numeric

For some of the models all the independent attribute should be of type numeric and Linear Regression model is one among them.
But this data set has some categorial attributes.

'pandas.get_dummies' To convert convert categorical variable into dummy/indicator variables


In [18]:
print (data.columns)

Index(['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')


Creating dummy variables.

    If we have k levels in a category, then we create k-1 dummy variables as the last one would be redundant. So we use the parameter drop_first in pd.get_dummies function that drops the first level in each of the category


In [19]:
categorical_Attributes = data.select_dtypes(include=['category']).columns

In [20]:
data = pd.get_dummies(columns=categorical_Attributes, data=data, prefix=categorical_Attributes, prefix_sep="_",
                      drop_first=True)

In [21]:
print (data.columns, data.shape)

Index(['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'pieces_past_due', 'perf_6_month_avg', 'perf_12_month_avg',
       'local_bo_qty', 'potential_issue_Yes', 'deck_risk_Yes',
       'oe_constraint_Yes', 'ppap_risk_Yes', 'stop_auto_buy_Yes',
       'rev_stop_Yes', 'went_on_backorder_Yes'],
      dtype='object') (9433, 22)


#### Target attribute distribution

In [22]:
pd.value_counts(data['went_on_backorder_Yes'])

0    7632
1    1801
Name: went_on_backorder_Yes, dtype: int64

#### Split the data in to train and test

sklearn.model_selection.train_test_split

    Split arrays or matrices into random train and test subsets

In [23]:
#Performing train test split on the data


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)


In [24]:
#To get the distribution in the target in train and test
print(pd.value_counts(y_train))
print(pd.value_counts(y_test))

0    5372
1    1231
dtype: int64
0    2260
1     570
dtype: int64


#### Perceptron Model Building

In [25]:
perceptron_model = Sequential()

perceptron_model.add(Dense(1, input_dim=21, activation='sigmoid', kernel_initializer='normal'))

In [26]:
perceptron_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
perceptron_model.fit(X_train, y_train, epochs=30, batch_size=64)

Epoch 1/30
6603/6603 [==============================] - 1s 120us/step - loss: 1.7499 - acc: 0.6128
Epoch 2/30
6603/6603 [==============================] - 0s 35us/step - loss: 0.5552 - acc: 0.8248
Epoch 3/30
6603/6603 [==============================] - 0s 44us/step - loss: 0.4539 - acc: 0.8340
Epoch 4/30
6603/6603 [==============================] - 0s 48us/step - loss: 0.5924 - acc: 0.8275
Epoch 5/30
6603/6603 [==============================] - 0s 35us/step - loss: 0.4049 - acc: 0.8401
Epoch 6/30
6603/6603 [==============================] - 0s 38us/step - loss: 0.4115 - acc: 0.8343
Epoch 7/30
6603/6603 [==============================] - 0s 41us/step - loss: 0.3880 - acc: 0.8460
Epoch 8/30
6603/6603 [==============================] - 0s 39us/step - loss: 0.3739 - acc: 0.8481
Epoch 9/30
6603/6603 [==============================] - 0s 35us/step - loss: 0.3694 - acc: 0.8490
Epoch 10/30
6603/6603 [==============================] - 0s 38us/step - loss: 0.3645 - acc: 0.8466
Epoch 11/30
6603/6

#### Predictions

In [28]:
y_pred=perceptron_model.predict_classes(X_test)
y_train_pred=perceptron_model.predict_classes(X_train)

#### Getting evaluation metrics and evaluating model performance

In [29]:
print("Train data target \n", pd.value_counts(y_train))
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
print("\n Confusion matrix \n", confusion_matrix_train)

print("\n Test data target \n", pd.value_counts(y_test))
confusion_matrix_test = confusion_matrix(y_test, y_pred)
print("\n Confusion matrix \n", confusion_matrix_test)

Train data target 
 0    5372
1    1231
dtype: int64

 Confusion matrix 
 [[5301   71]
 [ 841  390]]

 Test data target 
 0    2260
1     570
dtype: int64

 Confusion matrix 
 [[2231   29]
 [ 393  177]]


#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [30]:
Accuracy_Train=(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+
                                                                          confusion_matrix_train[0,1]+
                                                                          confusion_matrix_train[1,0]+
                                                                          confusion_matrix_train[1,1])
TNR_Train= confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train= confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train)
print("Train TPR: ",TPR_Train)
print("Train Accuracy: ",Accuracy_Train)

Train TNR:  0.986783320923306
Train TPR:  0.31681559707554835
Train Accuracy:  0.8618809631985461


In [100]:
Accuracy_Test=(confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test= confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test= confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test)
print("Test TPR: ",TPR_Test)
print("Test Accuracy: ",Accuracy_Test)

Test TNR:  0.959292035398
Test TPR:  0.519298245614
Test Accuracy:  0.870671378092
